### Проверка и настройка форматов изображений

#### Поиск изображений, не соответствующих формату

In [9]:
import os
from PIL import Image

def check_image_sizes(folder_path, target_size=(224, 224), save_report=True):
    """
    Проверяет размеры изображений в указанной папке и её подпапках.
    
    Параметры:
        folder_path (str): Путь к папке с изображениями
        target_size (tuple): Ожидаемый размер (ширина, высота)
        save_report (bool): Сохранять ли отчёт в файл
    """
    if not os.path.exists(folder_path):
        print(f"Ошибка: Папка '{folder_path}' не существует!")
        return
    
    wrong_sizes = []
    supported_formats = ('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.webp')
    
    print(f"\n Проверка изображений в папке: {folder_path}")
    print(f"Ожидаемый размер: {target_size[0]}x{target_size[1]} пикселей\n")
    
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(supported_formats):
                file_path = os.path.join(root, file)
                try:
                    with Image.open(file_path) as img:
                        width, height = img.size
                        if width != target_size[0] or height != target_size[1]:
                            wrong_sizes.append({
                                'path': file_path,
                                'size': f"{width}x{height}",
                                'mode': img.mode
                            })
                except Exception as e:
                    print(f" Ошибка при открытии {file_path}: {str(e)}")
    
    if wrong_sizes:
        print(f" Найдено {len(wrong_sizes)} изображений с некорректным размером:")
        print("\nПримеры проблемных файлов (первые 10):")
        for i, item in enumerate(wrong_sizes[:10]):
            print(f"{i+1}. {item['path']}")
            print(f" Размер: {item['size']}, Цветовой режим: {item['mode']}")
    
    else:
        print("Все изображения имеют корректный размер!")
    
    # Статистика
    print("\n" + "=" * 50)
    total_images = sum(1 for _ in os.walk(folder_path) for f in _[2] if f.lower().endswith(supported_formats))
    print(f"Всего проверено файлов: {len(wrong_sizes)} с ошибками из {total_images} изображений")

if __name__ == "__main__":
    target_folder = "C:/Users/User/.cache/kagglehub/datasets/darshanthakare/dog-breeds-image-dataset/versions/1/cropped_dogs"  
    check_image_sizes(target_folder, target_size=(224, 224))


 Проверка изображений в папке: C:/Users/User/.cache/kagglehub/datasets/darshanthakare/dog-breeds-image-dataset/versions/1/cropped_dogs
Ожидаемый размер: 224x224 пикселей

Все изображения имеют корректный размер!

Всего проверено файлов: 0 с ошибками из 10117 изображений


#### Форматирование исходных изображений

In [6]:
import os
from PIL import Image
from tqdm import tqdm

def resize_images(folder_path, target_size=(224, 224), quality=95):
    """
    Рекурсивно изменяет размер всех изображений в папке
    :param folder_path: Путь к папке с изображениями
    :param target_size: Кортеж (ширина, высота)
    :param quality: Качество сохранения (1-100)
    """
    supported_formats = ('.jpg', '.jpeg', '.png', '.webp')
    
    for root, _, files in os.walk(folder_path):
        for file in tqdm(files, desc=f"Обработка {os.path.basename(root)}"):
            if file.lower().endswith(supported_formats):
                file_path = os.path.join(root, file)
                try:
                    with Image.open(file_path) as img:
                        if img.size != target_size:
                            # Сохраняем режим (RGB/CMYK) и EXIF данные
                            original_mode = img.mode
                            exif = img.info.get('exif')
                            
                            # Изменяем размер с сохранением пропорций (без растяжения)
                            img.thumbnail(target_size, Image.LANCZOS)
                            
                            # Создаем новое изображение нужного размера
                            new_img = Image.new("RGB", target_size, (255, 255, 255))
                            new_img.paste(img, ((target_size[0]-img.size[0])//2, 
                                              (target_size[1]-img.size[1])//2))
                            
                            new_img.save(file_path, 
                                       quality=quality, 
                                       exif=exif if exif else b'')
                except Exception as e:
                    print(f"Ошибка при обработке {file_path}: {str(e)}")

resize_images("C:/Users/User/.cache/kagglehub/datasets/darshanthakare/dog-breeds-image-dataset/versions/1/cropped_dogs", target_size=(224, 224))

Обработка cropped_dogs: 0it [00:00, ?it/s]
Обработка yorkshire_terrier: 100%|██████████| 131/131 [00:00<00:00, 429.72it/s]
